In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.optimize import minimize

In [3]:
aapl = yf.download(tickers='AAPL', period='1mo')
jnj = yf.download(tickers='JNJ', period='1mo')
jpm = yf.download(tickers='JPM', period='1mo')
pg = yf.download(tickers='PG', period='1mo')
xom = yf.download(tickers='XOM', period='1mo')
nee = yf.download(tickers='NEE', period='1mo')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
aapl['Return'] = aapl['Adj Close'].pct_change()
jnj['Return'] = jnj['Adj Close'].pct_change()
jpm['Return'] = jpm['Adj Close'].pct_change()
pg['Return'] = pg['Adj Close'].pct_change()
xom['Return'] = xom['Adj Close'].pct_change()
nee['Return']=nee['Adj Close'].pct_change()
returns = pd.DataFrame({
    'AAPL': aapl['Return'],
    'JNJ': jnj['Return'],
    'JPM': jpm['Return'],
    'PG': pg['Return'],
    'XOM': xom['Return'],
    'NEE':nee['Return']
})

# Drop any rows with NaN values
returns.dropna(inplace=True)

# Calculate the variance-covariance matrix
C = returns.cov()

print(C)


              AAPL       JNJ       JPM            PG       XOM       NEE
AAPL  4.493282e-04 -0.000044 -0.000140  5.714955e-07 -0.000096 -0.000189
JNJ  -4.369454e-05  0.000039  0.000016  6.104390e-06  0.000018  0.000034
JPM  -1.402028e-04  0.000016  0.000146 -7.297959e-06  0.000056  0.000066
PG    5.714955e-07  0.000006 -0.000007  5.499244e-05 -0.000014  0.000030
XOM  -9.557176e-05  0.000018  0.000056 -1.397895e-05  0.000127  0.000063
NEE  -1.894302e-04  0.000034  0.000066  3.045417e-05  0.000063  0.000401


In [5]:
#METHOD- 1 IMPLEMTATION

w0 = np.ones(6)/6   #initial guess for weight matrix
C = np.array(C)
def obj_function(w, C):
    P = C @ w
    MCR = P/np.sqrt(w.T @ C @ w)
    f = 0.0
    for i in range(len(w)):
        for j in range(len(w)):
            f += (w[i] * MCR[i] - w[j] * MCR[j]) ** 2
    return f

def objective(w):
    return obj_function(w, C)

# Constraints
constraints = (
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Equality constraint: sum(w) = 1
    {'type': 'ineq', 'fun': lambda w: w}             # Inequality constraint: w >= 0
)

# Bounds
bounds = [(0, 1) for _ in range(C.shape[1])]



# Optimize
result = minimize(objective, w0, method='SLSQP', constraints=constraints, bounds=bounds, options={'disp': True})

# Optimal weights
optimal_weights = result.x

print("Optimal weights:", optimal_weights)
print("Objective function value at optimal weights:", result.fun)
print("Success:", result.success)
print("Message:", result.message)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.654618020559084e-06
            Iterations: 6
            Function evaluations: 42
            Gradient evaluations: 6
Optimal weights: [0.20164891 0.17146279 0.16542257 0.17402397 0.166752   0.12068976]
Objective function value at optimal weights: 6.654618020559084e-06
Success: True
Message: Optimization terminated successfully


In [6]:
#METHOD-2 IMPLEMNTATION
w = np.ones(6)/6   #initial guess for weight matrix
C = np.array(C)
def obj_function(w, C):
    return np.sqrt(w.T @ C @ w)


def objective(w):
    return obj_function(w, C)

# Constraints
constraints = (
 {'type': 'eq', 'fun': lambda w: np.sum(np.log(w)) + 2 },  #I have taken c = -2, you can take other arbitrary constant which gives a feasible soln
    {'type': 'ineq', 'fun': lambda w: w}             # Inequality constraint: w >= 0
)

# Bounds
bounds = [(0, 1) for _ in range(C.shape[1])]



# Optimize
result = minimize(objective, w, method='SLSQP', constraints=constraints, bounds=bounds, options={'disp': True})

# Optimal weights
optimal_weights = result.x

print("Optimal weights:", optimal_weights)
print("Objective function value at optimal weights:", result.fun)
print("Success:", result.success)
print("Message:", result.message)

y = np.array(optimal_weights)
#Normalise
w = y/np.sum(y)
print(w)   #Final weight matrix

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.01777314510674372
            Iterations: 23
            Function evaluations: 162
            Gradient evaluations: 23
Optimal weights: [0.77823562 1.         0.7585469  0.96715905 0.66749654 0.35511615]
Objective function value at optimal weights: 0.01777314510674372
Success: True
Message: Optimization terminated successfully
[0.17192672 0.22091859 0.16757711 0.21366342 0.1474624  0.07845176]


In [7]:
# No. of days taken here are 105 which is equivalent to 5 months counted towards the working days in NASDAQ or NSE
expected_return=np.sum(returns[returns>0].mean()*w*105)
variance=np.dot(w.T,np.dot(returns[returns>0].cov()*105,w))



In [10]:
# No. of days taken here are 105 which is equivalent to 5 months counted towards the working days in NASDAQ or NSE
new_expected_return=np.sum(returns[returns>0].mean()*optimal_weights*105)
new_variance=np.dot(optimal_weights.T,np.dot(returns[returns>0].cov()*105,optimal_weights))



In [12]:
import numpy as np
import pandas as pd

# Market capitalization data (in billions)
market_caps = pd.Series({
    'AAPL': 2870,
    'JNJ': 425,
    'JPM': 460,
    'PG': 350,
    'XOM': 450,
    'NEE': 140
})


# Calculate market cap weights
total_market_cap = market_caps.sum()
market_cap_weights = market_caps / total_market_cap

# Calculate portfolio variance for market cap weighted portfolio
market_cap_portfolio_variance = np.dot(market_cap_weights.T, np.dot(C, market_cap_weights))


print("Market Cap Weighted Portfolio Variance:", market_cap_portfolio_variance)
print("Optimal Variance of Portfolio",variance)

Market Cap Weighted Portfolio Variance: 0.00013426323039636762
Optimal Variance of Portfolio 0.0014720528358540042
